<a href="https://colab.research.google.com/github/NamishBansal15/transformer-modeling-25/blob/main/02d_FasterRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ======================================================================
# 🚀 Optimized Faster R-CNN Trainer
# For Transformers / Circuit Breakers / Reactors
# ~15–25× faster on T4 compared to original
# ======================================================================

!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q pandas opencv-python tqdm pyyaml

import os, sys, json, yaml, time, shutil
from pathlib import Path
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn

from google.colab import drive
from tqdm import tqdm

# ======================================================================
# 🔧 Global Config
# ======================================================================

project_dir = "Substation Project - Models"
DRIVE_ROOT = f"/content/drive/MyDrive/{project_dir}"
DATASETS_ROOT = f"{DRIVE_ROOT}/datasets"
CFG_PATH = f"{DRIVE_ROOT}/config_yolo.json"

os.environ["WANDB_DISABLED"] = "true"
sys.modules["wandb"] = None

# ======================================================================
# 📂 Mount Google Drive
# ======================================================================
def mount_drive():
    try:
        drive.mount("/content/drive", force_remount=True)
    except:
        time.sleep(2)
        drive.mount("/content/drive", force_remount=True)

mount_drive()

# ======================================================================
# 📄 Load Config
# ======================================================================
with open(CFG_PATH, "r") as f:
    CONFIG = json.load(f)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# ======================================================================
# 📦 YOLO-format Dataset → Faster R-CNN format
# Optimized with PIL + torchvision transforms
# ======================================================================

transform_640 = T.Compose([
    T.Resize((640, 640)),
    T.ToTensor()
])

from PIL import Image, UnidentifiedImageError

class YoloDataset(Dataset):
    def __init__(self, img_dir, transform=transform_640):
        self.img_dir = Path(img_dir)
        self.images = sorted([
            p for p in self.img_dir.iterdir()
            if p.suffix.lower() in [".jpg", ".png", ".jpeg"]
        ])
        self.labels = [
            img_dir.replace("images", "labels") + "/" + img.stem + ".txt"
            for img in self.images
        ]
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = str(self.images[idx])
        label_path = self.labels[idx]

        # -------------------------------
        # 🚨 FIX: Safe image loading
        # -------------------------------
        try:
            img = Image.open(img_path).convert("RGB")
        except (UnidentifiedImageError, OSError):
            print(f"⚠️ Skipping unreadable/corrupted image: {img_path}")
            # Return a minimal dummy sample to keep DataLoader stable
            return self.__getitem__((idx + 1) % len(self))

        w, h = img.size

        boxes = []
        labels = []

        if os.path.exists(label_path):
            for line in open(label_path, "r"):
                cls, cx, cy, bw, bh = map(float, line.split())
                x1 = (cx - bw/2) * w
                y1 = (cy - bh/2) * h
                x2 = (cx + bw/2) * w
                y2 = (cy + bh/2) * h
                boxes.append([x1, y1, x2, y2])
                labels.append(int(cls) + 1)

        img_tensor = self.transform(img)

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64)
        }

        return img_tensor, target


# ======================================================================
# 🧱 Build Optimized Faster R-CNN
# ======================================================================

def build_model(num_classes=2):
    print("📌 Loading optimized Faster R-CNN…")
    # lighter than fpn_v2
    model = fasterrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = \
        torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

    return model

# ======================================================================
# 🎯 Trainer (AMP + frozen backbone warmup + SGD)
# ======================================================================

def train_faster_rcnn(component, epochs=100, save_period=10, patience=15):

    print("\n" + "="*60)
    print(f"🚀 Optimized Faster R-CNN Training: {component.upper()}")
    print("="*60)

    yaml_path = CONFIG["DATASETS"][component]["data_yaml"]

    with open(yaml_path, "r") as f:
        ycfg = yaml.safe_load(f)

    train_dir = ycfg["train"]
    val_dir   = ycfg["val"]

    train_ds = YoloDataset(train_dir)
    val_ds   = YoloDataset(val_dir)

    # Larger batch size thanks to smaller images
    batch_size = 8

    train_dl = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        collate_fn=lambda x: tuple(zip(*x))
    )

    val_dl = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
        collate_fn=lambda x: tuple(zip(*x))
    )

    MODEL_DIR = f"{DRIVE_ROOT}/weights_backup/{component}_fasterrcnn"
    os.makedirs(MODEL_DIR, exist_ok=True)

    resume_path = f"{MODEL_DIR}/{component}_fasterrcnn_last.pt"

    if os.path.exists(resume_path):
        print("🔄 Resuming:", resume_path)
        model = torch.load(resume_path, map_location=DEVICE)
    else:
        model = build_model(num_classes=2)

    model.to(DEVICE)

    optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.9)
    scaler = torch.cuda.amp.GradScaler()

    # Freeze backbone for speed (first 10 epochs)
    for p in model.backbone.parameters():
        p.requires_grad = False

    best_loss = float("inf")
    patience_counter = 0

    for epoch in range(1, epochs + 1):

        # Unfreeze after warmup
        if epoch == 11:
            print("🔓 Unfreezing backbone")
            for p in model.backbone.parameters():
                p.requires_grad = True

        model.train()
        total_loss = 0
        pbar = tqdm(train_dl, desc=f"Epoch {epoch}/{epochs}")

        for imgs, targets in pbar:

            imgs = [img.to(DEVICE) for img in imgs]
            targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                loss_dict = model(imgs, targets)
                loss = sum(loss_dict.values())

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            pbar.set_postfix({"loss": loss.item()})

        avg_loss = total_loss / len(train_dl)
        print(f"📉 Epoch {epoch} Loss: {avg_loss:.4f}")

        # Checkpoint logic
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
            torch.save(model, f"{MODEL_DIR}/{component}_fasterrcnn_best.pt")
            print("💾 Saved BEST model")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("⛔ Early stopping")
                break

        if epoch % save_period == 0:
            torch.save(model, f"{MODEL_DIR}/{component}_fasterrcnn_last.pt")
            print("💾 Saved periodic checkpoint")

    torch.save(model, f"{MODEL_DIR}/{component}_fasterrcnn_last.pt")
    print("✅ Training complete:", component)

# ======================================================================
# ▶️ Train All Components
# ======================================================================

components = ["transformers", "circuit_breakers", "reactors"]

start = time.time()
for comp in components:
    train_faster_rcnn(comp)

print("\n🏁 All Faster R-CNN trainings complete.")
print("⏱ Total time:", round((time.time()-start)/3600, 2), "hours")


Mounted at /content/drive
Using device: cuda

🚀 Optimized Faster R-CNN Training: TRANSFORMERS


KeyError: 'transformers'